In [1]:
import zipfile
import pandas as pd

In [2]:
with zipfile.ZipFile('609158_1090983_compressed_clickbait_data.csv.zip') as f:
    df = pd.read_csv(f.open('clickbait_data.csv'))

In [3]:
df = df.head(5)
df.head()

,headline,clickbait
0,Should I Get Bings,1
1,Which TV Female Friend Group Do You Belong In,1
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1
3,"This Vine Of New York On ""Celebrity Big Brothe...",1
4,A Couple Did A Stunning Photo Shoot With Their...,1


In [4]:
import spacy
nlp = spacy.load('en_core_web_md')
stopwords = spacy.lang.en.STOP_WORDS

In [5]:
df.headline = list(nlp.pipe(df.headline))

In [6]:
df.head()

,headline,clickbait
0,"(Should, I, Get, Bings)",1
1,"(Which, TV, Female, Friend, Group, Do, You, Be...",1
2,"(The, New, "", Star, Wars, :, The, Force, Awake...",1
3,"(This, Vine, Of, New, York, On, "", Celebrity, ...",1
4,"(A, Couple, Did, A, Stunning, Photo, Shoot, Wi...",1


In [8]:
text = '18 photos single dads cannot understand!'
doc = nlp(text)

In [9]:
for token in doc:
    print(token.lemma_, token.pos_, token.tag_)

18 NUM CD
photo NOUN NNS
single ADJ JJ
dad NOUN NNS
can VERB MD
not PART RB
understand VERB VB
! PUNCT .


In [10]:
[token.text.lower() for token in doc]

['18', 'photos', 'single', 'dads', 'can', 'not', 'understand', '!']

In [5]:
from spacy.matcher import Matcher

In [6]:
matcher = Matcher(nlp.vocab)

In [7]:
pattern1 = [{'IS_DIGIT':True}, {'POS':'NOUN'}, {'POS':'ADJ'}]
pattern2 = [{'POS':'NUM'}, {'POS':'NOUN'}, {'POS':'ADJ'}]
# pattern3 = [{'IS_DIGIT':True}, {'POS':'NOUN'}, {'POS':'ADJ'}]

In [17]:
matcher.add('list_title', None, pattern)

NameError: name 'pattern' is not defined

In [18]:
# matches = matcher(df.headline)

In [19]:
def matcher(doc):
    doc_ = nlp(doc)
    matcher_ = Matcher(nlp.vocab)
    pattern_1 = [{'IS_DIGIT':True}, {'POS':'NOUN'}, {'POS':'ADJ'}]
    pattern_2 = [{'POS':'NUM'}, {'POS':'NOUN'}, {'POS':'ADJ'}]
    matcher_.add('list_title', None, pattern_1, pattern_2)
    matches_ = matcher_(doc_)

    if matches_ == []:
        return 'No match'
    else:
        for matchId, start, end in matches_:
            string_id = nlp.vocab.strings[matchId]
            span = doc_[start:end]
            print(matchId, string_id, start, end, span.text)
            return 'Match'
#             return matchId
#     return 'Match'

In [87]:
df['list'] = df['headline'].apply(matcher)

11971384090149641623 list_title 0 3 17 Pictures Hot
11971384090149641623 list_title 0 3 17 Things Only
11971384090149641623 list_title 0 3 14 Fall Themed
11971384090149641623 list_title 0 3 17 Truths Only
11971384090149641623 list_title 0 3 19 Things Only
11971384090149641623 list_title 5 8 10 Years Old
11971384090149641623 list_title 0 3 20 Problems Only
11971384090149641623 list_title 0 3 21 Things New
11971384090149641623 list_title 0 3 12 Things Salvadoran
11971384090149641623 list_title 0 3 15 Things Single
11971384090149641623 list_title 0 3 21 Things Military
11971384090149641623 list_title 0 3 19 Things American
11971384090149641623 list_title 0 3 10 Things Spanish
11971384090149641623 list_title 0 3 12 Things Black
11971384090149641623 list_title 0 3 22 Things Scottish
11971384090149641623 list_title 0 3 19 Things Lazy
11971384090149641623 list_title 6 9 20 Years Old
11971384090149641623 list_title 13 16 4 Kids Ready
11971384090149641623 list_title 8 11 Two Decades Old
1197138

In [88]:
a = matcher('18 photos single dads cannot understand!')
print(a)

11971384090149641623 list_title 0 3 18 photos single
Match


In [89]:
# df.headline = df.headline.apply(lambda row: [token for token in row if not token.is_stop])

In [86]:
df.head()

,headline,clickbait
0,Should I Get Bings,1
1,Which TV Female Friend Group Do You Belong In,1
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1
3,"This Vine Of New York On ""Celebrity Big Brothe...",1
4,A Couple Did A Stunning Photo Shoot With Their...,1


In [28]:
df.headline = df.headline.apply(lambda row: [token.lemma_ for token in row])

In [20]:
df

,headline,clickbait
0,"(Should, I, Get, Bings)",1
1,"(Which, TV, Female, Friend, Group, Do, You, Be...",1
2,"(The, New, "", Star, Wars, :, The, Force, Awake...",1
3,"(This, Vine, Of, New, York, On, "", Celebrity, ...",1
4,"(A, Couple, Did, A, Stunning, Photo, Shoot, Wi...",1


In [30]:

from string import punctuation
df.headline = df.headline.apply(lambda row: [token for token in row if not token in punctuation])

In [31]:
df.head()

,headline,clickbait
0,"[Should, -PRON-, get, bing]",1
1,"[which, tv, Female, Friend, Group, do, -PRON-,...",1
2,"[the, New, Star, Wars, the, Force, awaken, tra...",1
3,"[this, vine, of, New, York, on, Celebrity, Big...",1
4,"[a, couple, do, a, stunning, Photo, shoot, wit...",1


In [32]:
df['headline'][4]

['a',
 'couple',
 'do',
 'a',
 'stunning',
 'Photo',
 'shoot',
 'with',
 '-PRON-',
 'baby',
 'after',
 'Learning',
 '-PRON-',
 'have',
 'an',
 'inoperable',
 'brain',
 'tumor']

In [4]:
type(df.headline[0])

str

In [8]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 15 13:34:25 2020

@author: Shanding Gershinen
"""


import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from string import punctuation
import spacy

nlp = spacy.load('en_core_web_md')
stopwords = spacy.lang.en.STOP_WORDS



In [9]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()
# Create a blank Tokenizer with just the English vocab
tokenizer = Tokenizer(nlp.vocab)

In [28]:

class TextTransformer(BaseEstimator, TransformerMixin):
    """
    Input:
        dict, DataFrame
    
    Output:
        DataFrame
    
        
    Clean text data.
    Performs mulltiple transformation on a dataframe.
    
    The input to this transformer should be a dictionary ('key' : 'Value') or a 
    DataFrame containing string characcters to be transformed. Lemmentization and 
    stemming and stopwords removal are among the transformations applied to the 
    DataFrame. This creates a new DataFrame with containing cleaned text.
    
    By default, this transformer performs:
        - Removes punctuation marks present in a string or document.
        - Remove stopwords from the string or document.
        - Lemmentize the string or document. 
    After successfull compleetion of the transformation, a new DataFrame is 
    returned.
    
    The dictionary fed to the transformer MUST have a 'title' and 'text' keys 
    while a DataFrame must contain 'text' and 'title' columns. 
    
    
    Examples
    --------
    Given a dataset with two features, we let the transformer clean and transform
    the data into a clean DataFrame.
    
    tt = TextTransformer()
    X = [['Male', 1], ['Female', 3], ['Female', 2]]
    tt.fit(X)

    """

    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        
        return self
    
    def transform(self, X):
        Xt = X.copy()
        if not isinstance(Xt, pd.DataFrame):
            Xt = pd.DataFrame(Xt)
        Xt.headline = list(nlp.pipe(Xt.headline))
        
#       Remove stop words from the text data
        Xt.headline = Xt.headline.apply(lambda row: [token.text for token in row if not token.is_stop])

#       Lower text
        Xt.headline = Xt.headline.apply(lambda row: [token.lower() for token in row])

#       Remove punctuations from the title title and text columns
        Xt.headline = Xt.headline.apply(lambda row: nlp(' '.join([token for token in row if not token in punctuation])))


#       Lemmatize the text data
        Xt.headline = Xt.headline.apply(lambda row: ' '.join([token.lemma_ for token in row]))
    
        
        
        
        return Xt


In [29]:
clean = TextTransformer()

In [30]:
c = clean.fit_transform(df)

In [31]:
c

,headline,clickbait
0,bing,1
1,tv female friend group belong,1
2,new star war force awaken trailer chill,1
3,vine new york celebrity big brother fuck perfect,1
4,couple stun photo shoot baby learn inoperable ...,1


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
v = TfidfVectorizer()

In [19]:
v.fit_transform(X_train)

<4x42 sparse matrix of type '<class 'numpy.float64'>'
	with 44 stored elements in Compressed Sparse Row format>

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                        random_state=42)

In [34]:
X = c.headline
y = c.clickbait.values

In [137]:
q = v.fit_transform(X_train)

q

<4x42 sparse matrix of type '<class 'numpy.float64'>'
	with 44 stored elements in Compressed Sparse Row format>

In [123]:
# lr.fit(q, y_train)

In [110]:
y_train.reshape(-1, 1).shape


(4, 1)

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [21]:
lr = LogisticRegression()

In [32]:
pipe = Pipeline(steps=[
    ('clean', clean), 
    ('vect', v),
    ('lr', lr)
])

In [33]:
pipe.fit(X_train, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [1, 4]

In [36]:
X_train.shape

(4,)

In [37]:
y_train.shape

(4,)

In [38]:
x = v.fit_transform(X_train)
x.shape

(4, 24)

In [ ]:
xx = v.fit_transform()

In [39]:
lr.fit(x, y_train)

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1